In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [1]:
import ee
import geemap

In [3]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [4]:
# Add Earth Engine dataset
def subsetNAIP(img_col, startTime, endTime, fc):
    img = img_col.filterDate(startTime, endTime).filterBounds(fc).mosaic().clip(fc)
    return img

def calNDWI(image, threshold):
    """A function to compute NDWI."""
    ndwi = image.normalizedDifference(['G', 'N'])
    ndwiViz = {'min': 0, 'max': 1, 'palette': ['00FFFF', '0000FF']}
    ndwiMasked = ndwi.updateMask(ndwi.gte(threshold))
    ndwi_bin = ndwiMasked.gt(0)
    patch_size = ndwi_bin.connectedPixelCount(500, True)
    large_patches = patch_size.eq(500)
    large_patches = large_patches.updateMask(large_patches)
    opened = large_patches.focal_min(1).focal_max(1)
    return opened

def rasterToVector(img, fc):
    vec = img.reduceToVectors(geometry=fc, eightConnected=True, maxPixels=59568116121, crs=img.projection(), scale=1)
    return vec

def exportToDrive(vec, filename):
    taskParams = {
        'driveFolder': 'image',
        'fileFormat': 'KML'
    }
    task = ee.batch.Export.table(vec, filename, taskParams)
    task.start()


years = [2014]
threshold = 0.3

collection = ee.ImageCollection('USDA/NAIP/DOQQ')
polys = aoi
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
values = fromFT.reduceColumns(ee.Reducer.toList(2), ['system:index', 'name']).getInfo()['list']

vis = {'bands': ['N', 'R', 'G']}

for year in years:
    startTime = ee.Date(str(year) + '-01-01')
    endTime = ee.Date(str(year) + '-12-31')
    for (id, name) in values:
        watershed = fromFT.filter(ee.Filter.eq('system:index', str(id)))
        filename = "Y" + str(year) + "_" + str(id) + "_" + str(name).replace(" ", "_")
        print(filename)
        image = subsetNAIP(collection, startTime, endTime, watershed)
        ndwi = calNDWI(image, threshold)
        vector = rasterToVector(ndwi, watershed)
        exportToDrive(vector, filename)



EEException: ignored